In [ ]:
# https://stable-baselines3.readthedocs.io/en/master/guide/rl.html
# https://spinningup.openai.com/en/latest/spinningup/rl_intro2.html#a-taxonomy-of-rl-algorithms

# 1. Import dependencies

In [1]:
!pip install stable-baselines3[extra]

In [3]:
import os
import gym 
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy
from tqdm import tqdm

# 2. Load Environment

In [4]:
environment_name = "CartPole-v0"

In [5]:
env = gym.make(environment_name)

In [ ]:
!pip install pyglet

In [6]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

Episode:1 Score:12.0
Episode:2 Score:12.0
Episode:3 Score:40.0
Episode:4 Score:18.0
Episode:5 Score:48.0


# Understanding The Environment
https://github.com/openai/gym/blob/master/gym/envs/classic_control/cartpole.py

In [7]:
# 0-push cart to left, 1-push cart to the right
env.action_space.sample()

1

In [8]:
# [cart position, cart velocity, pole angle, pole angular velocity]
env.observation_space.sample()

array([ 2.7092966e-01, -4.4323951e+37,  1.9755277e-01, -3.2199826e+38],
      dtype=float32)

# 3. Train an RL Model

In [9]:
log_path = os.path.join('Training','Logs')

In [10]:
env = gym.make(environment_name)
env = DummyVecEnv([lambda: env])
model = PPO('MlpPolicy', env, verbose = 1)

Using cpu device


In [11]:
model.learn(total_timesteps=20000)

-----------------------------
| time/              |      |
|    fps             | 2729 |
|    iterations      | 1    |
|    time_elapsed    | 0    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 1733        |
|    iterations           | 2           |
|    time_elapsed         | 2           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.009118118 |
|    clip_fraction        | 0.101       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.686      |
|    explained_variance   | 0.000517    |
|    learning_rate        | 0.0003      |
|    loss                 | 7.12        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0153     |
|    value_loss           | 54.9        |
-----------------------------------------
----------------------------------

# 4. Save and Reload Model

In [12]:
PPO_path = os.path.join('Training', 'Saved Models', 'PPO_model')

In [13]:
model.save(PPO_path)

In [ ]:
del model

In [ ]:
model = PPO.load('PPO_model', env=env)

# 4. Evaluation

In [14]:
from stable_baselines3.common.evaluation import evaluate_policy

In [15]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

D:\ReinforcementLearning\rl\lib\site-packages\stable_baselines3\common\evaluation.py:65: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


(200.0, 0.0)

In [16]:
env.close()

# 5. Test Model

In [17]:
obs = env.reset()
while True:
    action, _states = model.predict(obs)
    obs, rewards, done, info = env.step(action)
    env.render()
    if done: 
        print('info', info)
        break

info [{'TimeLimit.truncated': True, 'terminal_observation': array([ 0.14756502,  0.33927318, -0.06138154, -0.47982508], dtype=float32)}]


In [21]:
model.predict(obs)

(array([0], dtype=int64), None)

In [18]:
env.close()

# 6. Viewing Logs in Tensorboard

In [19]:
training_log_path = os.path.join(log_path, 'PPO_3')

In [20]:
!tensorboard --logdir={training_log_path}

^C


# 7. Adding a callback to the training Stage

In [22]:
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold
import os

In [23]:
save_path = os.path.join('Training', 'Saved Models')
log_path = os.path.join('Training', 'Logs')

In [24]:
env = gym.make(environment_name)
env = DummyVecEnv([lambda: env])

In [25]:
stop_callback = StopTrainingOnRewardThreshold(reward_threshold=190, verbose=1)
eval_callback = EvalCallback(env, 
                             callback_on_new_best=stop_callback, 
                             eval_freq=10000, 
                             best_model_save_path=save_path, 
                             verbose=1)

In [26]:
model = PPO('MlpPolicy', env, verbose = 1, tensorboard_log=log_path)

Using cpu device


In [27]:
model.learn(total_timesteps=20000, callback=eval_callback)

Logging to Training\Logs\PPO_1
-----------------------------
| time/              |      |
|    fps             | 2844 |
|    iterations      | 1    |
|    time_elapsed    | 0    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 1768        |
|    iterations           | 2           |
|    time_elapsed         | 2           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.009442848 |
|    clip_fraction        | 0.113       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.686      |
|    explained_variance   | 0.00504     |
|    learning_rate        | 0.0003      |
|    loss                 | 7.05        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0163     |
|    value_loss           | 52.3        |
-----------------------------------------
---

In [28]:
model_path = os.path.join('Training', 'Saved Models', 'best_model')
model = PPO.load(model_path, env=env)

In [29]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

(200.0, 0.0)

In [30]:
env.close()

# 8. Changing Policies

In [31]:
net_arch=[dict(pi=[128, 128, 128, 128], vf=[128, 128, 128, 128])]

In [32]:
model = PPO('MlpPolicy', env, verbose = 1, policy_kwargs={'net_arch': net_arch})

Using cpu device


In [33]:
model.learn(total_timesteps=20000, callback=eval_callback)

-----------------------------
| time/              |      |
|    fps             | 2241 |
|    iterations      | 1    |
|    time_elapsed    | 0    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 1174        |
|    iterations           | 2           |
|    time_elapsed         | 3           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.014541181 |
|    clip_fraction        | 0.205       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.681      |
|    explained_variance   | -0.00283    |
|    learning_rate        | 0.0003      |
|    loss                 | 3.99        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0231     |
|    value_loss           | 20.2        |
-----------------------------------------
----------------------------------

# 9. Using an Alternate Algorithm

In [34]:
from stable_baselines3 import DQN

In [35]:
model = DQN('MlpPolicy', env, verbose = 1, tensorboard_log=log_path)

Using cpu device


In [36]:
model.learn(total_timesteps=20000, callback=eval_callback)

Logging to Training\Logs\DQN_1
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.966    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 8873     |
|    time_elapsed     | 0        |
|    total_timesteps  | 71       |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.927    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 10263    |
|    time_elapsed     | 0        |
|    total_timesteps  | 154      |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.901    |
| time/               |          |
|    episodes         | 12       |
|    fps              | 10397    |
|    time_elapsed     | 0        |
|    total_timesteps  | 208      |
----------------------------------
------------------------

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 108      |
|    fps              | 12628    |
|    time_elapsed     | 0        |
|    total_timesteps  | 2326     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 112      |
|    fps              | 12627    |
|    time_elapsed     | 0        |
|    total_timesteps  | 2389     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 116      |
|    fps              | 12688    |
|    time_elapsed     | 0        |
|    total_timesteps  | 2502     |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 216      |
|    fps              | 12710    |
|    time_elapsed     | 0        |
|    total_timesteps  | 4771     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 220      |
|    fps              | 12669    |
|    time_elapsed     | 0        |
|    total_timesteps  | 4819     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 224      |
|    fps              | 12678    |
|    time_elapsed     | 0        |
|    total_timesteps  | 4924     |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 324      |
|    fps              | 12697    |
|    time_elapsed     | 0        |
|    total_timesteps  | 7092     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 328      |
|    fps              | 12716    |
|    time_elapsed     | 0        |
|    total_timesteps  | 7179     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 332      |
|    fps              | 12700    |
|    time_elapsed     | 0        |
|    total_timesteps  | 7246     |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 432      |
|    fps              | 12710    |
|    time_elapsed     | 0        |
|    total_timesteps  | 9456     |
----------------------------------
Eval num_timesteps=9520, episode_reward=9.60 +/- 0.80
Episode length: 9.60 +/- 0.80
----------------------------------
| eval/               |          |
|    mean_ep_length   | 9.6      |
|    mean_reward      | 9.6      |
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    total_timesteps  | 9520     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 436      |
|    fps              | 12522    |
|    time_elapsed     | 0        |
|    total_timesteps  | 9542     |
------

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 532      |
|    fps              | 12514    |
|    time_elapsed     | 0        |
|    total_timesteps  | 11643    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 536      |
|    fps              | 12511    |
|    time_elapsed     | 0        |
|    total_timesteps  | 11716    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 540      |
|    fps              | 12499    |
|    time_elapsed     | 0        |
|    total_timesteps  | 11779    |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 640      |
|    fps              | 12497    |
|    time_elapsed     | 1        |
|    total_timesteps  | 13871    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 644      |
|    fps              | 12499    |
|    time_elapsed     | 1        |
|    total_timesteps  | 13960    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 648      |
|    fps              | 12504    |
|    time_elapsed     | 1        |
|    total_timesteps  | 14079    |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 748      |
|    fps              | 12506    |
|    time_elapsed     | 1        |
|    total_timesteps  | 16232    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 752      |
|    fps              | 12512    |
|    time_elapsed     | 1        |
|    total_timesteps  | 16315    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 756      |
|    fps              | 12510    |
|    time_elapsed     | 1        |
|    total_timesteps  | 16388    |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 856      |
|    fps              | 12502    |
|    time_elapsed     | 1        |
|    total_timesteps  | 18548    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 860      |
|    fps              | 12498    |
|    time_elapsed     | 1        |
|    total_timesteps  | 18618    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 864      |
|    fps              | 12507    |
|    time_elapsed     | 1        |
|    total_timesteps  | 18718    |
----------------------------------
----------------------------------
| rollout/          

In [37]:
dqn_path = os.path.join('Training', 'Saved Models', 'DQN_model')

In [38]:
model.save(dqn_path)

In [39]:
model = DQN.load(dqn_path, env=env)

In [40]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

(9.6, 0.4898979485566356)

In [41]:
env.close()